<a href="https://colab.research.google.com/github/Reshoan/Excel-Summary/blob/main/Excel_Summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
%%writefile app.py

from pyngrok import ngrok
import streamlit as st
import pandas as pd
from io import BytesIO
import re
import time
from openpyxl import load_workbook
import base64

# ------ STREAMLIT APP ------ #

st.title("Excel Form Processor")

# Step 1: Upload Excel file
uploaded_file = st.file_uploader("Upload an Excel file", type=["xlsx"])

if uploaded_file:
    xls = pd.ExcelFile(uploaded_file)

    # Must contain formFields
    if "formFields" not in xls.sheet_names:
        st.error("The Excel file must contain a sheet named 'formFields'.")
    else:

        df = pd.read_excel(uploaded_file, sheet_name="formFields")

        # Fixed required columns
        field_name_col = "name"
        field_type_col = "input_type"
        field_order_col = "order"
        form_col = "formName"

        # Fixed category columns
        fixed_category_cols = [
            "is_editable", "is_hidden", "is_required", "is_nullable",
            "is_multiselect", "is_richtext", "editor_height",
            "is_encrypted", "is_conditional"
        ]
        selected_category_cols = [c for c in fixed_category_cols if c in df.columns]

        # Load fieldMapping sheet
        field_mapping_df = pd.DataFrame(columns=["formName", "fieldName"])
        if "fieldMapping" in xls.sheet_names:
            field_mapping_df = pd.read_excel(uploaded_file, sheet_name="fieldMapping")

        # Process all unique forms
        all_forms = sorted(df[form_col].dropna().unique())

        # --- TOTAL sheet count = form sheets + Transition_mapping ---
        total_sheet_count = len(all_forms) + 1

        st.write(f"Processing {total_sheet_count} sheets automatically...")

        # Live-updating numbered sheet list
        sheet_list_placeholder = st.empty()

        output = BytesIO()
        progress_bar = st.progress(0)

        uploaded_file.seek(0)
        wb = load_workbook(uploaded_file)

        # Remove existing transition mapping sheet if exists
        if "Transition_mapping" in wb.sheetnames:
            wb.remove(wb["Transition_mapping"])

        created_sheet_names = []  # list to show sheets live

        # --------------------------- #
        # Generate form-based sheets
        # --------------------------- #
        for i, full_form_name in enumerate(all_forms):

            parts = full_form_name.split("_")
            if len(parts) > 3:
                middle_parts = parts[2:-1]
                short_name = "_".join(middle_parts)
            else:
                short_name = full_form_name

            safe_short_name = re.sub(r'[\\/*?:\[\]]', '_', short_name)
            safe_short_name = safe_short_name[:31]

            form_df = df[df[form_col] == full_form_name].sort_values(by=field_order_col)

            final_df = form_df[[field_name_col, field_type_col, field_order_col]].copy()

            # Repeater column
            repeater_list = []
            filtered_mapping = field_mapping_df[field_mapping_df['formName'] == full_form_name]
            mapped_fields = set(filtered_mapping['fieldName'].dropna())

            for field in final_df[field_name_col]:
                repeater_list.append("Yes" if field in mapped_fields else "")

            final_df.insert(len(final_df.columns), "Repeater", repeater_list)

            # Categories column
            if selected_category_cols:
                categories_list = []
                for idx, row in form_df.iterrows():
                    selected = [col for col in selected_category_cols if row.get(col) == 1]
                    selected.sort()
                    categories_list.append(",".join(selected) if selected else "")
                final_df["Categories"] = categories_list

            # Remove existing sheet
            if safe_short_name in wb.sheetnames:
                wb.remove(wb[safe_short_name])

            # Create new sheet
            ws = wb.create_sheet(title=safe_short_name)

            # Write header A1 & B1
            ws.cell(row=1, column=1, value="Form")
            ws.cell(row=1, column=2, value=full_form_name)

            # Column headers row 2
            for c_idx, header in enumerate(final_df.columns, start=1):
                ws.cell(row=2, column=c_idx, value=header)

            # Data rows from row 3
            for r_idx, row in enumerate(final_df.values, start=3):
                for c_idx, value in enumerate(row, start=1):
                    ws.cell(row=r_idx, column=c_idx, value=value)

            # Track created sheet
            created_sheet_names.append(safe_short_name)

            # Live updating numbered sheet list
            numbered_list = [f"{idx+1}. {name}" for idx, name in enumerate(created_sheet_names)]
            sheet_list_placeholder.markdown(
                "<div style='max-height:200px; overflow-y:auto; border:1px solid #444; padding:10px; border-radius:5px;'>"
                + "<br>".join(numbered_list)
                + "</div>",
                unsafe_allow_html=True
            )

            # Progress update
            progress_bar.progress(int((len(created_sheet_names) / total_sheet_count) * 100))
            time.sleep(0.05)

        # ----------------------------------------------------------- #
        #                   TRANSITION MAPPING PART
        # ----------------------------------------------------------- #

        if "transitions" not in xls.sheet_names:
            st.error("No 'transitions' sheet found. Cannot build Transition_mapping.")
        else:
            transitions_df = pd.read_excel(uploaded_file, sheet_name="transitions")

            transitions_df = transitions_df[
                ["name", "workflowFromStateName", "workflowToStateName",
                 "workflowFormName", "fromToStateName"]
            ]

            # case-insensitive sort
            transitions_df = transitions_df.sort_values(
                by=["workflowFromStateName", "workflowToStateName"],
                key=lambda col: col.str.lower()
            )

            transitions_df["sla_time"] = ""
            transitions_df["sla_time_type"] = ""

            if "workflowSlas" in xls.sheet_names:
                slas_df = pd.read_excel(uploaded_file, sheet_name="workflowSlas")

                for idx, row in transitions_df.iterrows():
                    from_state = row["workflowFromStateName"]
                    to_state = row["workflowToStateName"]

                    match = slas_df[
                        (slas_df["workflowFromStateName"] == from_state) &
                        (slas_df["workflowToStateName"] == to_state)
                    ]

                    if not match.empty:
                        transitions_df.at[idx, "sla_time"] = match.iloc[0]["sla_time"]
                        transitions_df.at[idx, "sla_time_type"] = match.iloc[0]["sla_time_type"]

            transitions_df = transitions_df[
                ["name", "workflowFromStateName", "workflowToStateName",
                 "workflowFormName", "sla_time", "sla_time_type", "fromToStateName"]
            ]

            ws = wb.create_sheet("Transition_mapping")

            for c_idx, header in enumerate(transitions_df.columns, start=1):
                ws.cell(row=1, column=c_idx, value=header)

            for r_idx, row in enumerate(transitions_df.values, start=2):
                for c_idx, value in enumerate(row, start=1):
                    ws.cell(row=r_idx, column=c_idx, value=value)

            # Add Transition sheet to live list
            created_sheet_names.append("Transition_mapping")
            numbered_list = [f"{idx+1}. {name}" for idx, name in enumerate(created_sheet_names)]
            sheet_list_placeholder.markdown(
                "<div style='max-height:200px; overflow-y:auto; border:1px solid #444; padding:10px; border-radius:5px;'>"
                + "<br>".join(numbered_list)
                + "</div>",
                unsafe_allow_html=True
            )

            # Final progress update
            progress_bar.progress(100)

        # ----------------------------------------------------------- #

        wb.save(output)
        output.seek(0)
        # Get the original file name
        original_name = uploaded_file.name if uploaded_file is not None else "file.xlsx"
        download_name = f"Processed_{original_name}"

        # Encode the BytesIO data for download link
        b64 = base64.b64encode(output.getvalue()).decode()
        button_html = f'''
        <div style="margin-top:20px; margin-bottom:20px;">
            <a href="data:application/vnd.openxmlformats-officedocument.spreadsheetml.sheet;base64,{b64}" download="{download_name}">
                <button style="padding:10px 20px; font-size:16px; color:white; background-color:#4CAF50; border:none; border-radius:5px; cursor:pointer;">
                    Download Modified Excel
                </button>
            </a>
        </div>
        '''

        st.write("")
        st.success("Modified Excel created successfully and download should start automatically. Otherwise press the button bellow.")
        st.markdown(button_html, unsafe_allow_html=True)


Overwriting app.py


In [9]:
# Install packages (if not installed already)
!pip install streamlit pyngrok openpyxl --quiet

from pyngrok import ngrok
import time

# Kill previous tunnels
ngrok.kill()

# Set your ngrok token
ngrok.set_auth_token("35gXlCpbnwCAIJ9I2xiA1GXDu2u_3UoQf6QeGYhFC9MxYapgo")

# Start Streamlit in the background
get_ipython().system_raw("streamlit run app.py &")

# Wait a few seconds for Streamlit to start
time.sleep(5)

# Open ngrok tunnel
public_url = ngrok.connect(8501)
print("Streamlit app running at:", public_url)


Streamlit app running at: NgrokTunnel: "https://armless-twirly-alanna.ngrok-free.dev" -> "http://localhost:8501"
